# **Julia** workshop

![](JuliaLogo.jpeg)

# Level of this workshop: **Beginner**

### __(You are all welcome to work together and learn)__

> ## In this workshop, you will see:

- ### a brief background of Julia Language
- ### what are the advantages implemented in Julia
- ### some applications in chemistry
- ### some applications in machine learning
- ### a brief and fun hands-on
- ### how the julia community works

### The objective of this workshop is **not** to make you an expert in Julia, but to inspire you starting applying julia in your projects, studies or research!

![](JuliaHistory.png)

### Julia language is a compiled programming language released in 2012!

### **Curious fact** 

#### Why the programming language is called Julia?

#### Some good points about Julia

![](Paper.png)

- ### Speed 
(why is it important for chemistry ?)

#### **Extra topic**: run a benchmarking (Python vs. Julia)

- ### Syntax

##### 1.2. **Print** method 

In [1]:
print("Julia is a cool language!")

Julia is a cool language!

### Now it is your **turn to practice**!

**Activity 1** : Print your own name.

**Activity 2** : Make the computer spell your name with a ```for``` loop.

- ### Multiple dispatch

In [4]:
function number(N::Int64)
    print(N)
end

number (generic function with 1 method)

In [5]:
print(5)

5

> There is support for ```Int16```, ```Int32```, ```Int64```. (The difference is the uage of bits to allocate it). In the data structures in julia, all these types are subtypes of ```Integer```.

In [6]:
function number(N::T) where T <: Integer
    print(N)
end

number (generic function with 2 methods)

> For example, let's think about one example that is more reasonable: a ```Molecule``` is a subtype of ```ChemicalSystem```, right? So, let us build it in Julia!

In [7]:
abstract type ChemicalSystem end ### this is a supertype.

In [8]:
struct Molecule <: ChemicalSystem end

In [9]:
function number(N::Float64)
    print(N)
end

number (generic function with 3 methods)

In [10]:
number(5.2)

5.2

### **Practice** an example on REPL! >> Let's do it together!

- print ```integer``` if Int; 
- print ```float``` if Float.

### **Math** notation

In [14]:
f(x) = x+2 ## this is an expression

f (generic function with 1 method)

In [15]:
f(2)

4

> Julia also support UNICODES! (what is so fun!)

In [16]:
f(θ) = α + β

f (generic function with 1 method)

## 2. How to use libraries in Julia

#### Use **Pkg** manager

In [17]:
using Pkg

##### **Extra topic**: Let's learn how to use Pkg on **REPL (Read-Eval-Print Loop)**

In [18]:
Pkg.add("Molly")

InterruptException: InterruptException:

## Applications in Chemistry/Materials Science/Physics

### Our **first simulation**

#### Fluid in a Lennard-Jones potential

In [19]:
using Molly

InterruptException: InterruptException:

In [20]:
n_atoms = 100
atom_mass = 10.0u"u"
atoms = [Atom(mass=atom_mass, σ=0.3u"nm", ϵ=0.2u"kJ * mol^-1") for i in 1:n_atoms]

LoadError: LoadError: UndefVarError: @u_str not defined
in expression starting at /Users/leticiamadureira/RIIA_workshop/HandsOn.ipynb:2

In [21]:
boundary = CubicBoundary(2.0u"nm", 2.0u"nm", 2.0u"nm") # Periodic boundary conditions
coords = place_atoms(n_atoms, boundary; min_dist=0.3u"nm") # Random placement without clashing

temp = 100.0u"K"
velocities = [velocity(atom_mass, temp) for i in 1:n_atoms]

LoadError: LoadError: UndefVarError: @u_str not defined
in expression starting at /Users/leticiamadureira/RIIA_workshop/HandsOn.ipynb:1

In [22]:
pairwise_inters = (LennardJones(),) # Don't forget the trailing comma!

UndefVarError: UndefVarError: LennardJones not defined

In [23]:
sys = System(
    atoms=atoms,
    pairwise_inters=pairwise_inters,
    coords=coords,
    velocities=velocities,
    boundary=boundary,
    loggers=(
        temp=TemperatureLogger(10),
        coords=CoordinateLogger(10),
    ),
)

simulator = VelocityVerlet(
    dt=0.002u"ps",
    coupling=AndersenThermostat(temp, 1.0u"ps"),
)

simulate!(sys, simulator, 1_000)

UndefVarError: UndefVarError: TemperatureLogger not defined

In [24]:
using GLMakie
visualize(sys.loggers.coords, boundary, "sim_lj.mp4")

### Molly also supports GPU acceleration

In [ ]:
using Pkg

#### Do not forget to install the packages using ```Pkg.install("CUDA")```

In [ ]:
using CUDA

In [ ]:
n_atoms = 100
atom_mass = 10.0f0u"u"
boundary = CubicBoundary(2.0f0u"nm", 2.0f0u"nm", 2.0f0u"nm")
temp = 100.0f0u"K"
atoms = CuArray([Atom(mass=atom_mass, σ=0.3f0u"nm", ϵ=0.2f0u"kJ * mol^-1") for i in 1:n_atoms])
coords = CuArray(place_atoms(n_atoms, boundary; min_dist=0.3u"nm"))
velocities = CuArray([velocity(atom_mass, temp) for i in 1:n_atoms])
simulator = VelocityVerlet(dt=0.002f0u"ps")

sys = System(
    atoms=atoms,
    pairwise_inters=(LennardJones(),),
    coords=coords,
    velocities=velocities,
    boundary=boundary,
    loggers=(
        temp=TemperatureLogger(typeof(1.0f0u"K"), 10),
        coords=CoordinateLogger(typeof(1.0f0u"nm"), 10),
    ),
)

simulate!(sys, simulator, 1_000)

### Let us simulate a diatomic molecule

In [ ]:
coords = place_atoms(n_atoms ÷ 2, boundary; min_dist=0.3u"nm")
for i in 1:length(coords)
    push!(coords, coords[i] .+ [0.1, 0.0, 0.0]u"nm")
end

velocities = [velocity(atom_mass, temp) for i in 1:n_atoms]

#### Simulating the protein

### A **fun topic**: simulating the solar system

In [ ]:
using GLMakie
using Molly
ß
# Using get_body_barycentric_posvel from Astropy
coords = [
    SVector(-1336052.8665050615,  294465.0896030796 ,  158690.88781384667)u"km",
    SVector(-58249418.70233503 , -26940630.286818042, -8491250.752464907 )u"km",
    SVector( 58624128.321813114, -81162437.2641475  , -40287143.05760552 )u"km",
    SVector(-99397467.7302648  , -105119583.06486066, -45537506.29775053 )u"km",
    SVector( 131714235.34070954, -144249196.60814604, -69730238.5084304  )u"km",
]

velocities = [
    SVector(-303.86327859262457, -1229.6540090943934, -513.791218405548  )u"km * d^-1",
    SVector( 1012486.9596885007, -3134222.279236384 , -1779128.5093088674)u"km * d^-1",
    SVector( 2504563.6403826815,  1567163.5923297722,  546718.234192132  )u"km * d^-1",
    SVector( 1915792.9709661514, -1542400.0057833872, -668579.962254351  )u"km * d^-1",
    SVector( 1690083.43357355  ,  1393597.7855017239,  593655.0037930267 )u"km * d^-1",
]

body_masses = [
    1.989e30u"kg",
    0.330e24u"kg",
    4.87e24u"kg" ,
    5.97e24u"kg" ,
    0.642e24u"kg",
]

boundary = CubicBoundary(1e9u"km", 1e9u"km", 1e9u"km")

# Convert the gravitational constant to the appropriate units
inter = Gravity(G=convert(typeof(1.0u"km^3 * kg^-1 * d^-2"), Unitful.G))

sys = System(
    atoms=[Atom(mass=m) for m in body_masses],
    pairwise_inters=(inter,),
    coords=coords .+ (SVector(5e8, 5e8, 5e8)u"km",),
    velocities=velocities,
    boundary=boundary,
    loggers=(coords=CoordinateLogger(typeof(1.0u"km"), 10),),
    force_units=u"kg * km * d^-2",
    energy_units=u"kg * km^2 * d^-2",
)

simulator = Verlet(
    dt=0.1u"d",
    remove_CM_motion=false,
)

simulate!(sys, simulator, 3650) # 1 year

visualize(
    sys.loggers.coords,
    boundary,
    "sim_planets.mp4";
    trails=5,
    color=[:yellow, :grey, :orange, :blue, :red],
    markersize=[0.25, 0.08, 0.08, 0.08, 0.08],
    transparency=false,
)


### Julia for Machine Learning

- Recommending packages

- Showing tutorials

### Conclusion: **Julia is a super powerful language**